In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import time
import os

In [5]:
from process import gen_second_data, get_dist_matrix

In [6]:
FOLDER = '../../data_handson/champs-scalar-coupling/'
os.listdir(FOLDER)

['scalar_coupling_contributions.csv',
 'mulliken_charges.csv',
 'structures.csv',
 'test.csv',
 'structures.zip',
 'train.csv',
 'magnetic_shielding_tensors.csv',
 'dipole_moments.csv',
 'sample_submission.csv',
 'potential_energy.csv']

## load all files as pandas dataframe

In [10]:
# df_mulliken_charges = pd.read_csv(FOLDER + 'mulliken_charges.csv')
# df_sample =  pd.read_csv(FOLDER + 'sample_submission.csv')
# df_magnetic_shielding_tensors = pd.read_csv(FOLDER + 'magnetic_shielding_tensors.csv')
df = pd.read_csv(FOLDER + 'train.csv')
# df_test = pd.read_csv(FOLDER + 'test.csv')
# df_dipole_moments = pd.read_csv(FOLDER + 'dipole_moments.csv')
# df_potential_energy = pd.read_csv(FOLDER + 'potential_energy.csv')
df_structures = pd.read_csv(FOLDER + 'structures.csv')
# df_scalar_coupling_contributions = pd.read_csv(FOLDER + 'scalar_coupling_contributions.csv')

In [11]:
df_structures_idx = df_structures.set_index('molecule_name')
df_idx = df.set_index('molecule_name')

In [15]:
mols = df['molecule_name'].unique()

In [25]:
m = mols[100]
atom_idx = df_idx.loc[m].query('type == "1JHC"')['atom_index_0'].values[0]
df_idx.loc[m][df_idx.loc[m]['atom_index_0'] == atom_idx]

,id,atom_index_0,atom_index_1,type,scalar_coupling_constant
molecule_name,,,,,
dsgdb9nsd_000144,2740,5,0,3JHN,0.736124
dsgdb9nsd_000144,2741,5,1,2JHC,5.759870
dsgdb9nsd_000144,2742,5,2,1JHC,126.926000
dsgdb9nsd_000144,2743,5,3,2JHC,-1.751980
dsgdb9nsd_000144,2744,5,6,3JHH,2.257290
dsgdb9nsd_000144,2745,5,7,3JHH,0.305795


In [26]:
atom_idx = df_idx.loc[m].query('type == "1JHC"')['atom_index_1'].values[0]
df_idx.loc[m][df_idx.loc[m]['atom_index_1'] == atom_idx]

,id,atom_index_0,atom_index_1,type,scalar_coupling_constant
molecule_name,,,,,
dsgdb9nsd_000144,2742,5,2,1JHC,126.926000
dsgdb9nsd_000144,2747,6,2,2JHC,-0.289389
dsgdb9nsd_000144,2751,7,2,2JHC,-1.715470


In [35]:
def pickup_bond_value_dist(df_mol_idx_0, dist_arr, bond, target_col):
    df_mol_idx_b = df_mol_idx_0.query('type == "{}"'.format(bond))
    atoms_b = df_mol_idx_b['atom_index_1'].values
    dist_b = dist_arr[atoms_b]
    predicts_b = df_mol_idx_b[target_col]        
    sorting_b = np.argsort(dist_b)
    return predicts_b.values[sorting_b], 1/dist_arr[atoms_b][sorting_b]

def gen_second_data(df_idx, df_structures_idx, m, target_bond='1JHC', target_col='scalar_coupling_constant'):
    try:
        type(df_idx.loc[m]) == pd.Series
    except:
        print("series exception:", m)
        return
    dist_mat = get_dist_matrix(df_structures_idx, m)    

    df_mol = df_idx.loc[m]
    con_id = df_mol.query('type == "{}"'.format(target_bond))['id'].values
    df_mol_idx = df_mol.set_index('id')

    bonds = ['1JHC', '1JHN', '2JHH', '2JHC', '2JHN', '3JHH', '3JHC', '3JHN']

    predict_01 = np.zeros([len(con_id), 2])
    features_0 = np.zeros([len(con_id), len(bonds)*3*2])
    features_1 = np.zeros([len(con_id), len(bonds)*3*2])

    for i, idx in enumerate(con_id):
        focus_0 = df_mol_idx.loc[idx]['atom_index_0']
        focus_1 = df_mol_idx.loc[idx]['atom_index_1']

        dist_arr = dist_mat[focus_0]
        predict_01[i, 0] = df_mol_idx.loc[idx][target_col]
        predict_01[i, 1] = 1/dist_arr[focus_1]

        df_mol_idx_0 = df_mol_idx.loc[df_mol_idx.index != idx].query('atom_index_0 == {}'.format(focus_0))
        for j, b in enumerate(bonds):
            predicts, inv_dist = pickup_bond_value_dist(df_mol_idx_0, dist_arr, b, target_col)
            if len(predicts) > 3:            
                features_0[i, j*3:(j+1)*3] = predicts[:3]
                features[i, (j+1)*3:(j+2)*3] = inv_dist[:3]
            else:
                features_0[i, j*3:j*3+len(predicts)] = predicts
                features_0[i, (j+1)*3:(j+1)*3+len(inv_dist)] = inv_dist
        
        df_mol_idx_1 = df_mol_idx.loc[df_mol_idx.index != idx].query('atom_index_1 == {}'.format(focus_1))
        for j, b in enumerate(bonds):
            predicts, inv_dist = pickup_bond_value_dist(df_mol_idx_1, dist_arr, b, target_col)
            if len(predicts) > 3:            
                features_1[i, j*3:(j+1)*3] = predicts[:3]
                features_1[i, (j+1)*3:(j+2)*3] = inv_dist[:3]
            else:
                features_1[i, j*3:j*3+len(predicts)] = predicts
                features_1[i, (j+1)*3:(j+1)*3+len(inv_dist)] = inv_dist
                
    features = np.hstack([predict_01, features_0, features_1])
    df_out = pd.DataFrame(features)
    df_out['id'] = con_id
    return df_out

In [54]:
m = mols[10]
bonds = ['1JHC', '1JHN', '2JHH', '2JHC', '2JHN', '3JHH', '3JHC', '3JHN']
df_sec = pd.DataFrame()

for b in bonds:
    df_sec_temp = gen_second_data(df_idx, df_structures_idx, m, target_bond=b)
    df_sec = pd.concat([df_sec, df_sec_temp], axis=0)

In [57]:
cols = df_sec.columns[df_sec.describe().loc['min'] == df_sec.describe().loc['max']]

In [62]:
for c in cols:
    del df_sec[c]

In [66]:
pd.merge(df_idx.loc[m], df_sec, on=['id'])

,id,atom_index_0,atom_index_1,type,scalar_coupling_constant,0,1,2,5,8,...,64,65,66,67,68,69,70,71,72,73
0,106,3,0,1JHC,83.51320,83.51320,0.912454,0.0000,0.000000,-11.4317,...,0.000000,0.00000,0.00000,0.00000,2.517980,10.439200,2.512820,0.912454,0.912454,0.912454
1,107,3,1,2JHC,-2.03004,-2.03004,0.458595,83.5132,0.912454,-11.4317,...,0.458595,0.00000,0.00000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,108,3,2,3JHC,2.51277,2.51277,0.354857,83.5132,0.912454,-11.4317,...,0.000000,0.00000,0.00000,0.00000,2.518020,10.439200,0.000000,0.354857,0.354857,0.000000
3,109,3,4,2JHH,-11.43170,-11.43170,0.566047,83.5132,0.912454,-11.4795,...,0.000000,0.00000,0.00000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,110,3,5,2JHH,-11.47950,-11.47950,0.565320,83.5132,0.912454,-11.4317,...,0.000000,0.00000,0.00000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,111,3,6,3JHH,3.42929,3.42929,0.395804,83.5132,0.912454,-11.4317,...,0.000000,12.33180,2.89449,0.00000,0.395804,0.395804,0.000000,0.000000,0.000000,0.000000
6,112,3,7,3JHH,12.33070,12.33070,0.324788,83.5132,0.912454,-11.4317,...,0.000000,3.42409,2.89907,0.00000,0.324788,0.324788,0.000000,0.000000,0.000000,0.000000
7,113,4,0,1JHC,83.51430,83.51430,0.912452,0.0000,0.000000,-11.4800,...,0.000000,0.00000,0.00000,0.00000,2.517980,10.439200,2.512820,0.912452,0.912452,0.912452
8,114,4,1,2JHC,-2.03044,-2.03044,0.458595,83.5143,0.912452,-11.4800,...,0.458595,0.00000,0.00000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,115,4,2,3JHC,2.51802,2.51802,0.354888,83.5143,0.912452,-11.4800,...,0.000000,0.00000,0.00000,0.00000,2.512770,10.439200,0.000000,0.354888,0.354888,0.000000
